# Instructions for Question 2

Using the provided Colab notebook and Yale Faces dataset, answer the following:

1. **Dataset Overview**  
   The Yale Faces dataset contains grayscale images of 15 subjects, each with six facial expressions. Each image has a resolution of 243 x 320 pixels.  
   - Preprocess the dataset by converting each image into a vector of size 77760 by stacking the columns of the image.  
   - Construct the PCA-representative image using only the first component to capture the central tendency of each subject's images.  

2. **Function Implementation**  
   Write a modular function named `predict_class` that meets the following specifications:  
   - **Inputs**:  
     - `image_number` (integer, 1–90): The number of the image to classify.  
     - `image_data` (NumPy array, 90 x 77760): Matrix representing the image vectors.  
     - `image_labels` (NumPy array, size 90): Labels corresponding to the image data.  
   - **Functionality**:  
     - Predict the subject number of the given image using the Euclidean distance between the input image vector and the PCA-representative images.  
     - Ensure all required library imports and preprocessing steps are modularized into separate functions or methods.  
   - **Output**:  
     - Return the `predicted_class` (integer) as the subject number.  

   Here is an example of the function signature and call:  

   ```python
   def predict_class(image_number, image_data, image_labels):
       # image_number: integer, from 1 to 90
       # image_data: NumPy array of shape 90 x 77760
       # image_labels: NumPy array of size 90
       
       # Include calls to your modular functions here
       
       return predicted_class
   ```

   Example test call:  
   ```python
   predicted_class = predict_class(64, image_data, labels)
   ```  

   **Expected Output**:  
   ```
   predicted_class = 11
   ```

---

### Notebook Organization:

- Ensure the notebook is **well-structured** and **modular**.
- Include all necessary **library imports** and **helper functions** in the earlier cells of the notebook.
- Write the final function, `predict_class(image_number, image_data, image_labels)`, in the **last cell** of the notebook.

---

### Submission Guidelines:

- Ensure your notebook is clean, with **appropriate comments** and **well-documented code**.  
- Verify that your function's output matches the expected values before submission.  
- Submit the notebook as a `.ipynb` file, ensuring it runs without errors.  



In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [1]:
!pip install rarfile

In [2]:
import numpy as np
import os
from scipy.spatial.distance import euclidean
import matplotlib.image as img
import imageio.v2 as iio
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import rarfile

In [3]:
# Extract files
rar_file_path = 'yalefacespng.rar'
image_directory = 'yalefacespng'
with rarfile.RarFile(rar_file_path) as rf:
  rf.extractall(image_directory)
  img_names = rf.namelist()
  img_names = sorted(img_names) # get list of image names

def preprocess_data(img_names):
  image_data = []

  for image_name in img_names:
    img_path = os.path.join(image_directory,image_name)
    img = Image.open(img_path).convert('L')  # Convert image to grayscale
    img_col = np.array(img)  # Convert to numpy array
    img_col = img_col.flatten()
    image_data.append(img_col)

  image_data = np.array(image_data)
  print(image_data.shape)
  image_labels = [file.split('.')[0] for file in img_names]
  return image_data,image_labels

# get image data and labels
image_data,image_labels = preprocess_data(img_names)

(90, 77760)


In [4]:
# get representative image data for each subject with the help of PCA
def get_sub_representative_image_arr(image_data):
  pca_repre_sub_img_arr = []

  for grp_idx in range(15):
    scaler = StandardScaler()
    img_matrix_per_group = image_data[grp_idx*6:grp_idx*6+6]
    # print([grp_idx*6,grp_idx*6+6])
    img_matrix_per_group = scaler.fit_transform(img_matrix_per_group)

    pca = PCA(n_components = 1)
    x_pca = pca.fit_transform(img_matrix_per_group)
    x_pca_original = pca.inverse_transform(x_pca)
    x_pca_reconstructed = scaler.inverse_transform(x_pca_original)
    pca_repre_sub_img_arr.append(x_pca_reconstructed[0]) # taking first component data only
    # print(x_pca_reconstructed.shape)

  pca_repre_sub_img_arr = np.array(pca_repre_sub_img_arr)
  return pca_repre_sub_img_arr

pca_repre_sub_img_arr = get_sub_representative_image_arr(image_data)
print(pca_repre_sub_img_arr.shape)


(15, 77760)


In [5]:
def predict_class(image_number, image_data, image_labels):
  pca_repre_sub_img_arr = get_sub_representative_image_arr(image_data) # get representative PCA array of subjects
  org_img_data = image_data[image_number-1] # get original data for image
  euclidean_dist_arr = []
  # get euclidean distance from each subject representative image
  for sub_idx in range(len(pca_repre_sub_img_arr)):
    euclidean_dist_arr.append(np.linalg.norm(pca_repre_sub_img_arr[sub_idx] - org_img_data))

  predicted_class = np.argmin(euclidean_dist_arr) + 1
  return predicted_class

In [6]:
image_number = 64
# call the function
predicted_class = predict_class(image_number, image_data, image_labels)
print('Predicted class is :',predicted_class)

Predicted class is : 11
